# PTMCMCSampler & Bayesian Analyses using the PTA Likelihood Function




In [1]:
import os, json, pickle

import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import scipy.stats as sps

import PTMCMCSampler.PTMCMCSampler as ptmcmc

from emcee.autocorr import integrated_time

import corner.corner as corner

Do not have mpi4py package.
Do not have acor package


`cloudpickle` is a Python package which allows dynamically constructed clases to be pickled, unlike `pickle`. `Enterprise` uses class factories to make different signal classes for each of the different pulsars in a PTA, hence in order to pickle the full PTA one needs `cloudpickle`. 

In [2]:
# !conda install -c conda-forge cloudpickle
# or install from PyPI
# !pip install cloudpickle

In [3]:
import cloudpickle

The easiest way to make the PTAs needed for this exercise is to make them with the script below. You will have to run the script on the command line to ensure that you use the correct ennvironment. 

``` Python
python mk_pta_pkls.py 
```

Check to see that the two pickle files have been saved in this directory. 

In [5]:
!ls vandy_3psr_fullpta_*.pkl

ls: vandy_3psr_fullpta_*.pkl: No such file or directory


If 2 different pickled files **don't** appear in this directory then you can alterantively copy/paste the script in `./mk_pta_pkls.py` in the cell below and delete the lines that pickle the 2 PTAs. We will need both loaded for the exercises below.

In [6]:
#!/usr/bin/env python
# coding: utf-8


import os, json, pickle

import numpy as np

from enterprise import constants as const
from enterprise.pulsar import Pulsar
from enterprise.signals import signal_base
from enterprise.signals import gp_signals
from enterprise.signals import gp_priors
from enterprise.signals import parameter
from enterprise.signals import selections

from enterprise_extensions import blocks

import cloudpickle

# # using `enterprise_extensions`
#
# `enterprise_extensions` provides "recipes" for commonly used functionality in `enterprise`.
#
# Lets build a 3 pulsar PTA that we could use to search for a gravitational wave background.

# ## load data

# In[2]:


# datafiles = {
#     "J1600-3053":{"par":"J1600-3053_EPTA_6psr.par", "tim":"J1600-3053_EPTA_6psr.tim"},
#     "J2241-5236":{"par":"J2241-5236_PPTA_dr2.par", "tim":"J2241-5236_PPTA_dr2.tim"},
#     "J2317+1439":{"par":"J2317+1439_NANOGrav_12y.par", "tim":"J2317+1439_NANOGrav_12y.tim"},
# }
#
# datadir = os.path.abspath("data")
#
#
# # In[3]:
#
#
# # load in each pulsar and append it to a list
# psrs = []
# for pname, fdict in datafiles.items():
#     pfile = os.path.join(datadir, fdict["par"])
#     tfile = os.path.join(datadir, fdict["tim"])
#     psrs.append(Pulsar(pfile, tfile))

datadir = os.path.abspath("../Day_1/data")

with open(datadir+'/viper_3psr.pkl','rb') as fin:
    psrs = pickle.load(fin)


# (once again, we can safely ignore these `tempo2` warnings)

# ## determine the PTA `Tspan`
# When building a `PTA` using data from multiple pulsars it helps to have a common Fourier basis for all of the pulsars' red noise (and common red noise, like GWB).  The easy way to do this is to use the total time-span of all data to set the Fourier frequencies.
#
# `enterprise.signals.gp_signals.FourierBasisGP` can use an intput `Tspan` to figure out the frequencies, and several functions in `enterprise_extensions` can too.

# In[4]:


# calculate the total Tspan
Tspan = np.max([pp.toas.max() - pp.toas.min() for pp in psrs])


# ## generate an enterprise `PTA` for all three pulsars for a CRN analysis
#
# Each pulsar needs a different noise model.  For CRN analysis it is common to fix the WN parameters based on previous single pulsar noise runs.
#
# To speed up the likelihood calculation we can use the `enterprise.signals.gp_signals.MarginalizingTimingModel`, which breaks the GP coefficient marginalization into two steps.  The linear timing model is analytically marginalized first.  This reduces the size of the matrices that must be inverted at each likelihood evaluation.  Only the Fourier Basis GPs (RN, DM, GWB, ...) contribute.
#
# We're going to use a spatially correlated common red noies model with a powerlaw spectrum as our GWB.
#
# Let's start by building the parts of the model that all pulsars will include:
#
# * timing model
# * red noise -- 30 frequency powerlaw -- `enterprise_extensions.blocks.red_noise_block`
# * GWB -- 15 frequency powerlaw, Hellings-Downs correlated -- `enterprise_extensions.blocks.common_red_noise_block`
#  * $\log_{10} A \rightarrow$ Uniform(-18, -13)
#  * $\gamma=13/3$

# In[6]:


# make the timing model signal
tm = gp_signals.MarginalizingTimingModel(use_svd=True)


# In[7]:


# make the RN signal
rn = blocks.red_noise_block(
    psd="powerlaw", components=30,
    Tspan=Tspan
)


# In[8]:


# make the GWB signal
gw = blocks.common_red_noise_block(
    psd="powerlaw", components=15,
    gamma_val=13/3,
    orf="hd",
    Tspan=Tspan
)

# make the Common Red Process signal
crn = blocks.common_red_noise_block(
    psd="powerlaw", components=15,
    gamma_val=13/3,
    orf=None,
    Tspan=Tspan
)


# Since each pulsar has a unique model, we'll store the three `SignalCollections` as a list.

# In[9]:


# empty list to store each pulsar's "signal" model
crn_sigs = []
gw_sigs = []



# ### generate an enterprise signal model for EPTA's J1600 pulsar
#
# In addition to the timing model, RN, and GWB, we need to include:
#
# * white noise -- fixed EFAC & EQUAD per backend (no ECORR)
# * DM variations -- 100 frequency powerlaw DM GP
#
# These are easy to do using `enterprise_extensions.blocks`.
#
# For a GWB analysis it is common to hold the white noise parameters (EFAC/EQUAD/ECORR) fixed to some known value (as determined by a single pulsar analysis.
# This reduces the number of parameters in the full PTA model.
# `enterprise` accomplishes this by using the `parameter.Constant` class.
# `enterprise_extensions.blocks.white_noise_block` has a boolean option to control this behavior.
# We'll use `vary=False` for **fixed** WN.

# In[10]:


# make the WN signal
wn = blocks.white_noise_block(vary=False, inc_ecorr=False, select="backend")


# In[11]:


# make the DM GP signal
dm = blocks.dm_noise_block(gp_kernel="diag", psd="powerlaw", components=100, Tspan=Tspan)


# In[12]:


# append J1600's SignalCOllection to the list
gw_sigs.append(tm + wn + rn + dm + gw)
crn_sigs.append(tm + wn + rn + dm + crn)


# ### generate an enterprise signal model for PPTA's J2241 pulsar
#
# In addition to the timing model, RN, and GWB, we need to include:
#
# * white noise -- fixed EFAC & EQUAD per backend (no ECORR)
# * DM variations -- 100 frequency powerlaw DM GP
# * band noise -- 30 frequency powerlaw in the 20cm band
#
# We can reuse the same `wn` and `dm` signals from before.
#
# To implement band noise we need a `enterprise.signal.selections.Selection`.
# A selection function takes the `dict` of TOA flags and flagvals as input.
# It returns a `dict` whose keys are the flagvals to select and mask (array of True/False) telling which TOAs have that flag.
#
# There's a built in `by_band` selection function, but that applies band noise to **all** bands.
# We only want to apply this model to TOAs in the 20cm band, so we need a selection function that returns a `dict` with one key and a mask for that flagval.

# In[13]:


def band_20cm(flags):
    """function to select TOAs in 20cm band (-B 20CM)"""
    flagval = "20CM"
    return {flagval: flags["B"] == flagval}

by_band_20cm = selections.Selection(band_20cm)


# There's no band noise block in `enterprise_extensions` but we can make a Fourier basis GP with the appropriate selection the old fashioned way!

# In[14]:


# band noise parameters
BN_logA = parameter.Uniform(-20, -11)
BN_gamma = parameter.Uniform(0, 7)

# band noise powerlaw prior
powlaw = gp_priors.powerlaw(log10_A=BN_logA, gamma=BN_gamma)

# make band noise signal (don't forget the name!)
bn = gp_signals.FourierBasisGP(
    powlaw,
    components=30,
    Tspan=Tspan,
    selection=by_band_20cm,
    name="band_noise"
)


# In[15]:


# append J2241's SignalCOllection to the list
gw_sigs.append(tm + wn + rn + bn + dm + gw)
crn_sigs.append(tm + wn + rn + bn + dm + crn)


# ### generate an enterprise signal model for NANOGrav's J2317 pulsar
#
# In addition to the timing model, RN, and GWB, we need to include:
#
# * white noise -- fixed EFAC, EQUAD, **and ECORR** per backend
#
# Remember there is no DM variations model, because DMX is already in the timing model for NANOGrav's 12.5yr data release

# In[16]:


# make WN signal (now with ECORR!)
wn_ec = blocks.white_noise_block(vary=False, inc_ecorr=True, select="backend")


# In[17]:


# append J2317's SignalCOllection to the list
gw_sigs.append(tm + wn_ec + rn + gw)
crn_sigs.append(tm + wn_ec + rn + crn)


# ## put the three pulsars together into a `PTA` object
#
# We can instantiate a PTA object with a list of three pulsar models.
# We simply feed each `Pulsar` to its `SignalCollection`, and then pass the whole list of instantiated models to `signal_base.PTA`.

# In[18]:


pta_gw = signal_base.PTA([ss(pp) for ss,pp in zip(gw_sigs, psrs)])
pta_crn = signal_base.PTA([ss(pp) for ss,pp in zip(crn_sigs, psrs)])

# ### load noise dictionary
#
# At this point we never actually told `enterprise` what to use for the fixed the WN parameters.
# We can use `PTA.set_default_params` to pass in the correct WN values from a `dict`.
#
# First we'll load the dictionary, which is stored as a `.json` file in the `data/` directory

# In[19]:


nfile = os.path.join(datadir, "viper_3psr_noise.json")
with open(nfile, "r") as f:
    noisedict = json.load(f)

# set the fixed WN params
pta_gw.set_default_params(noisedict)
pta_crn.set_default_params(noisedict)

with open('./vandy_3psr_fullpta_gwb.pkl','wb') as fout:
    cloudpickle.dump(pta_gw,fout)

with open('./vandy_3psr_fullpta_crn.pkl','wb') as fout:
    cloudpickle.dump(pta_crn,fout)


In [7]:
with open('./vandy_3psr_fullpta_crn.pkl','rb') as fin:
    pta_crn = cloudpickle.load(fin)
    
with open('./vandy_3psr_fullpta_gwb.pkl','rb') as fin:
    pta_gwb = cloudpickle.load(fin)

### test the log-likelihoods and log-priors!

In [8]:
# generate a random point
x0 = {pp.name:pp.sample() for pp in pta_gwb.params}

In [9]:
# calculate logL and logPr
print('GWB PTA: ',pta_gwb.get_lnlikelihood(x0), pta_gwb.get_lnprior(x0))
print('CRN PTA: ',pta_crn.get_lnlikelihood(x0), pta_crn.get_lnprior(x0))

GWB PTA:  225313.50361250053 -26.245102719469088
CRN PTA:  225313.50360684513 -26.245102719469088


## The Challenges of the PTA Likelihood Function
There are a number of reasons why it's challenging to sample your way to a reasonable posterior or evidence with starting from a PTA likelihood:

1. The continual growth of PTA datasets and the need to do full dataset cross-correlation calculations slows down the likelihood evalution. See Steve's talk from yesterday for scaling of the likelihood evaluation.
2. Large parameter spaces combined with slow likelihood evaluations considerably slows down the amount of time to convergence.
3. Lack of derivative information from `enterprise` functions makes the use of Hamiltonian Monte Carlo techniques challenging.
4. Always remember that things are this slow and we are _marginalizing over the timing model_ and _setting the white noise parameters constant_! (i.e. stay humble and keep looking for better ways to sample :-)

Please keep looking and testing new algorithms and samplers as the appear in the literature and in the open source code community!!

## Parameter Estimation

### MCMC

`enterprise_extensions.sampler.setup_sampler()` returns a `PTMCMCSampler` object.
The MCMC sampler can be tuned to improve performance.
The defaults of `setup_sampler` are often fine, but suboptimal.
One can fine improved acceptance and convergence, but adjusting the inputs to `setup_sampler` or setting up the sampler __manually__.

In [10]:
import enterprise_extensions.sampler as Sampler
from enterprise_extensions.sampler import JumpProposal as jp

In [11]:
pta_crn.param_names

['J1600-3053_dm_gp_gamma',
 'J1600-3053_dm_gp_log10_A',
 'J1600-3053_red_noise_gamma',
 'J1600-3053_red_noise_log10_A',
 'J2241-5236_band_noise_20CM_gamma',
 'J2241-5236_band_noise_20CM_log10_A',
 'J2241-5236_dm_gp_gamma',
 'J2241-5236_dm_gp_log10_A',
 'J2241-5236_red_noise_gamma',
 'J2241-5236_red_noise_log10_A',
 'J2317+1439_red_noise_gamma',
 'J2317+1439_red_noise_log10_A',
 'gw_log10_A']

In [12]:
ndim = len(pta_crn.param_names)
cov_new = np.diag(np.ones(ndim) * 0.1**2)  # param covariance, used for jump proposals
groups = [[0,1],[2,3],[0,1,2,3],[4,5],]
sampler0 = ptmcmc.PTSampler(ndim=ndim,  # necessary to setup
                            logl=pta_crn.get_lnlikelihood,  # necessary to setup
                            logp=pta_crn.get_lnprior,  # necessary to setup
                            cov=cov_new,  # necessary to setup
                            groups = groups,
                            outDir='chains0/')

`PTMCMCSampler` contains a few built-in jump proposals including:
```
AM => Adaptive Metroplis
SCAM => Single Component Adaptive Metroplis
DE => Differential Evolution
```
The first two use the covariance matrix, which is iteratively recalculated, to make predictions about various parameters. The groups are used by the `AM` proposals. 

In [13]:
JP = jp(pta_crn)
sampler0.addProposalToCycle(JP.draw_from_gwb_log_uniform_distribution, 40)

In [14]:
# generate an initial sample and run the sampler!
p0 = np.hstack(list(x0.values()))

In [16]:
B = 5_000
T = 2
Nsamp = 20_000

N = T * Nsamp

In [17]:
sampler0.sample(p0,
                Niter=N,
                thin=T,
                burn=B)

/Users/jacaseyclyde/miniconda3/envs/viper/lib/python3.9/site-packages/enterprise/signals/parameter.py:62: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 12.50 percent in 14.188779 s Acceptance rate = 0.21925Adding DE jump with weight 20
Finished 97.50 percent in 131.962667 s Acceptance rate = 0.251538
Run Complete


### `enterprise_extensions` sampler

Check the output to see what is generated and plot a trace of the log-Posterior

* set the output directory
* specify yourself as the "human" running the job

* remember the sampler takes a `numpy.ndarray` for the starting location, not a `dict`

* set `burn=5000` (the DE buffer) DO NOT confuse with the usual use of `burn`
* set `thin=2` (save every other sample)
* collect `20000` samples (if we thin by 2, we'll need to run for `Niter=40000`!)

In [18]:
# setup the sampler
sampler = Sampler.setup_sampler(pta_crn, outdir='crn_pta', human="jsh")

Adding red noise prior draws...

Adding DM GP noise prior draws...

Adding GWB uniform distribution draws...



In [19]:
sampler.sample(p0,
               Niter=N,
               burn=B,
               thin=T)

Finished 12.50 percent in 13.937409 s Acceptance rate = 0.3014Adding DE jump with weight 20
Finished 97.50 percent in 127.132059 s Acceptance rate = 0.322256
Run Complete


### Jump Proposals
One can add more jump proposals. The `JumpProposal` class is available to use for more various new proposals.

In [20]:
def draw_from_band_noise_prior(self, x, iter, beta):

        q = x.copy()
        lqxy = 0

        # draw parameter from signal model
        name = 'band_noise'
        par = np.random.choice([p for p in self.pnames if name in p])
        idx = list(self.pnames).index(par)
        param = self.params[idx]

        q[self.pmap[str(param)]] = np.random.uniform(param.prior._defaults['pmin'], param.prior._defaults['pmax'])

        # forward-backward jump probability
        lqxy = (param.get_logpdf(x[self.pmap[str(param)]]) -
                param.get_logpdf(q[self.pmap[str(param)]]))

        return q, float(lqxy)

In [21]:
sampler2 = Sampler.setup_sampler(pta_crn, outdir="crn_pta_jp", human="jsh")

Adding red noise prior draws...

Adding DM GP noise prior draws...

Adding GWB uniform distribution draws...



**Note:** Below we are changing the generic `Sampler` class not just an instance of the class by adding this new jump proposal as an attribute.

In [22]:
Sampler.JumpProposal.draw_from_band_noise_prior = draw_from_band_noise_prior

In [23]:
sampler2.addProposalToCycle(sampler2.jp.draw_from_band_noise_prior, 40) # Number is the relative weight of proposal

In [24]:
sampler2.jp.snames

{'dm_gp': [J1600-3053_dm_gp_log10_A:Uniform(pmin=-20, pmax=-11),
  J1600-3053_dm_gp_gamma:Uniform(pmin=0, pmax=7),
  J2241-5236_dm_gp_log10_A:Uniform(pmin=-20, pmax=-11),
  J2241-5236_dm_gp_gamma:Uniform(pmin=0, pmax=7)],
 'ecorr_sherman-morrison': [],
 'marginalizing linear timing model': [],
 'measurement_noise': [],
 'red noise': [J2241-5236_red_noise_log10_A:Uniform(pmin=-20, pmax=-11),
  gw_log10_A:Uniform(pmin=-18, pmax=-14),
  J2317+1439_red_noise_gamma:Uniform(pmin=0, pmax=7),
  J2241-5236_red_noise_gamma:Uniform(pmin=0, pmax=7),
  J2241-5236_band_noise_20CM_gamma:Uniform(pmin=0, pmax=7),
  J2317+1439_red_noise_log10_A:Uniform(pmin=-20, pmax=-11),
  J1600-3053_red_noise_log10_A:Uniform(pmin=-20, pmax=-11),
  J2241-5236_band_noise_20CM_log10_A:Uniform(pmin=-20, pmax=-11),
  J1600-3053_red_noise_gamma:Uniform(pmin=0, pmax=7)]}

In [25]:
sampler2.sample(p0,
                Niter=N,
                burn=B,
                thin=T)

Finished 12.50 percent in 14.066790 s Acceptance rate = 0.28525Adding DE jump with weight 20
Finished 97.50 percent in 142.800007 s Acceptance rate = 0.301795
Run Complete


### Empirical Distributions
_Empirical distributions_ are probablity density functions based on samples from a previous MCMC analysis. Here we will use the noise runs done bye Paul Baker to obtain the white noise parameters that are being set constant.

In [26]:
from enterprise_extensions.empirical_distr import make_empirical_distributions

In [28]:
import la_forge.slices as sl
import la_forge.core as co
import la_forge.diagnostics as dg

In [29]:
!tar -xzf viper_3psr_noiseruns.tgz

In [30]:
to_get = []

for psr in pta_crn.pulsars:
    to_get.append([par for par in pta_crn.param_names if psr in par and 'J1600-3053_dm_gp' not in par])
    
slc = sl.SlicesCore(slicedirs=pta_crn.pulsars,
                    pars2pull=to_get,params=[it for sub in to_get for it in sub ])

J2317+1439/chain_1.txt is loaded.



In [31]:
params2d = []
for ii in range(int(len(slc.params)//2)):
    params2d.append([slc.params[ii],slc.params[ii+1]])
    
params2d

[['J1600-3053_red_noise_gamma', 'J1600-3053_red_noise_log10_A'],
 ['J1600-3053_red_noise_log10_A', 'J2241-5236_band_noise_20CM_gamma'],
 ['J2241-5236_band_noise_20CM_gamma', 'J2241-5236_band_noise_20CM_log10_A'],
 ['J2241-5236_band_noise_20CM_log10_A', 'J2241-5236_dm_gp_gamma'],
 ['J2241-5236_dm_gp_gamma', 'J2241-5236_dm_gp_log10_A']]

In [32]:
make_empirical_distributions(pta_crn,
                             paramlist=params2d,# List of 2 element lists to make 2d emp dists
                             params=slc.params, #List of params in chain array
                             chain=slc.chain,
                             filename='viper_3psr_pta_emp_distr.pkl')

In [33]:
sampler3 = Sampler.setup_sampler(pta_crn, 
                                 outdir="crn_pta_ed", 
                                 human="jsh",
                                 empirical_distr='viper_3psr_pta_emp_distr.pkl')

Extending empirical distributions to priors...

Attempting to add empirical proposals...

Adding red noise prior draws...

Adding DM GP noise prior draws...

Adding GWB uniform distribution draws...



In [34]:
sampler3.sample(
    p0, Niter=N,
    burn=B, thin=T
)

Finished 12.50 percent in 16.391260 s Acceptance rate = 0.30865Adding DE jump with weight 20
Finished 97.50 percent in 128.510320 s Acceptance rate = 0.324436
Run Complete


## Product Space Sampling with the `HyperModel` submodule

[Product space sampling](https://academic.oup.com/mnras/article/455/3/2461/1069531) allows one to calculate Bayes factors without using the Savage-Dickey approximation or calculating evidences or posterior odds ratios. Multiple models are concatenated into one large HyperModel along with a model parameter which selects which model likelihood is evaluated in each iteration. The relative number of samples in for the `nmodel` parameter can be interpreted as the odds ratio between the models.

In [37]:
from enterprise_extensions.hypermodel import HyperModel

In [38]:
ptas = {0:pta_crn,1:pta_gwb}

In [39]:
hm = HyperModel(ptas)

**Note:** The `log_weights` flag can be used to put models on an even footing when one model is preferred over another.

In [40]:
hsampler = hm.setup_sampler(outdir='hypermodel',empirical_distr='viper_3psr_pta_emp_distr.pkl',human='jsh')

Extending empirical distributions to priors...

Adding empirical proposals...

Adding red noise prior draws...

Adding DM GP noise prior draws...

Adding GWB uniform distribution draws...

Adding gw param prior draws...

Adding nmodel uniform distribution draws...



/Users/jacaseyclyde/miniconda3/envs/viper/lib/python3.9/site-packages/numpy/lib/arraysetops.py:272: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


In [41]:
x0 = hm.initial_sample()
hsampler.sample(x0,
                Niter=N,
                burn=B,
                thin=T)

Finished 12.50 percent in 50.437612 s Acceptance rate = 0.3928Adding DE jump with weight 20
Finished 97.50 percent in 419.973111 s Acceptance rate = 0.400615
Run Complete


# Things to work on during the hack time

You don't need to do all of these, and you don't need to do them in order.  Pick one (or more) to try.
MCMC's can take a **long** time to run.  You may want to set up a notebook, then wait to run it overnight.

While the MCMC is running you can use the post processing notebook and read in the samples using `la_forge` to check on progress.
The number of samples it will take to have well converged posteriors will depend on the specific model.
Using these settings in `sampler.sample` will run for 100k samples, saving every tenth.
That may be enough...
* `Niter=100_000`
* `burn=20_000`
* `thin=10`
Models with more parameters will require more samples to converge.


## questions:

* What changes to the "vanilla" sampler can be made to get better convergence?
* Do the proposal acceptances change when new proposals are added into the mix?
* If a model is weighted in the `HyperModel` class how does one "unweight" the models to compare the odds ratios?


## ideas:

* Try parallel tempering on the same `CRN` model above and compare the Gelman-Rubin statistic and autocorrelation lengths given the same number of samples. 

* Use the `HyperModel` class to find the odds ratio for a common red process versus a model _without_ any common signal, correlated or otherwise. You will need to edit the script that makes `PTA` classes by building a PTA without a common signal. 

* Choose another sampler, e.g.,[dynesty](), [ultranest](), [zeus](https://zeus-mcmc.readthedocs.io/en/latest/), [emcee](https://emcee.readthedocs.io/en/stable/)... (**Note:** These will not work easily for full PTAs, but should work on the simplified examples used above.)

* Pull recent public data from your favorite PTA and set up a `PTA` object to perform an **analysis of your choice**.  Here are `.pkl` files containing a list of `enterprise` `Pulsar` objects for three recent data releases:
 * [EPTA 6PSR](https://drive.google.com/file/d/1MyZX7ox_8TlRUhgk47NirNYcWfEz5ron/view?usp=sharing)
 * [PPTA DR2](https://drive.google.com/file/d/1at5S_ydfqGV2x0PzF4eCO_BXhQjfamKX/view?usp=sharing)
 * [NANOGrav 12.5y](https://drive.google.com/file/d/1eWNLgPOm7mYKAt3LYY_YIb1i19_n03xD/view?usp=sharing)

## Parallel Tempering
`PTMCMCSampler` has [PT functionality](https://academic.oup.com/gji/article/196/1/357/585739) available, but one needs to use MPI (Message Passing Interface), which is not compatible with Jupyter notebooks, in order to use it. If your laptop has multiple cores, or you have access to a cluster you can try a PT run. First you need to write a Python script (hint: start by converting this notebook into a script using `jupyter nbconvert --to=script sampling_the_enterprise_likelihood.ipynb`.) Second you would use the following command 
```
mpiexec -np 4 python your_script.py
```
The `-np 4` tells MPI to use 4 cores. You can change this number if you'd like. When the sampler detectes that there is an MPI environment it will automatically initiate parallel tempering. I'd recommend setting the `sampler.sample` flag `writeHotChains=True`, which will write the hot chains to disk. This is useful for knowing that PT is actually working. 